# Error during ARIMA(X) Forecasting

In [1]:
import pandas as pd
import pmdarima as pm
import numpy as np
import copy
import pickle
import time

# path_to_data = '/hpi/fs00/share/MPSS2021BA1/data/'
path_to_data = '../data/'

starttime = time.time()
print('Start reading the input file.')

TRAIN=12
# Run for the one or the other
# read dict where train size is TRAIN
input_file = open(str(path_to_data) + 'dict_of_chunk_series_with_expanding_test_and_steady_train_error_chunk.pickle', 'rb')
dict_of_chunk_series_with_test_and_train = pickle.load(input_file)
input_file.close()

endtime = round(((time.time() - starttime) / 60), 5)
print('Reading of the input file completed after '+str(endtime)+' minutes.')

Start reading the input file.
Reading of the input file completed after 0.01267 minutes.


In [2]:
import numpy as np

chunk = '201358.0_220045.0_2151-11-05 15:52:00'
chunk_iteration = 28

current_train_list = dict_of_chunk_series_with_test_and_train[chunk][chunk_iteration]['TRAIN_LIST_MEDIAN']

# Issue 1: RuntimeWarning
# RuntimeWarning: divide by zero encountered in reciprocal
#   return np.roots(self.polynomial_reduced_ma)**-1
# /hpi/fs00/home/marius.michaelis/.local/lib/python3.8/site-packages/statsmodels/tsa/statespace/sarimax.py:1897: RuntimeWarning: divide by zero encountered in reciprocal
#   return np.roots(self.polynomial_reduced_ma)**-1

# We could ignore Issue 1 by telling numpy to ignore this warning
np.seterr(divide = 'ignore')
# np.seterr(divide = 'warn')

arima1 = pm.auto_arima(current_train_list, suppress_warnings=True, error_action='ignore')

# When ignoring Issue 1, pm.auto_arima() is completed
# However, the arima model has the parameter with_intercept set to False
# with_intercept is per default set to True; all other arima models seem to not have this parameter set to True
print(arima1)

 ARIMA(2,2,2)(0,0,0)[0]          


In [7]:
# Comparison with arima model for different chunk_iteration

chunk = '201358.0_220045.0_2151-11-05 15:52:00'
chunk_iteration = 27

current_train_list = dict_of_chunk_series_with_test_and_train[chunk][chunk_iteration]['TRAIN_LIST_MEDIAN']

arima2 = pm.auto_arima(current_train_list, suppress_warnings=True, error_action='ignore')
print(arima2)

 ARIMA(2,1,0)(0,0,0)[0] intercept


In [8]:
TEST=1
TRAIN=12
i=17
chunk_iteration=28

# Issue 2: ValueError
# ValueError: Input contains NaN, infinity or a value too large for dtype('float64').
# We do not understand the cause of this problem
# Option to deal with issue 2: Abort iteration for arima models that have with_intercept set to False
pd.Series(arima1.predict(TEST), index=[*range(i+TRAIN,i+TRAIN+TEST,1)], name="forecast_list_arima")

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [9]:
# Comparison with arima model for different chunk_iteration
TEST=1
TRAIN=12
i=16
chunk_iteration=27

pd.Series(arima2.predict(TEST), index=[*range(i+TRAIN,i+TRAIN+TEST,1)], name="forecast_list_arima")

28    94.598426
Name: forecast_list_arima, dtype: float64